In [3]:
# from google.colab import drive
import keras
from keras.models import load_model
import keras.layers as L
import numpy as np

In [4]:
filter_means = np.array([32, 32, 256, 256, 256, 512, 512, 512, 512])
filter_means_2 = filter_means

In [5]:
def get_conv_model(filter_means, input_layer):
  for i in range(len(filter_means)):
    if i == 0:
      conv = L.Conv1D(filters=filter_means[i], kernel_size=9, padding='same', strides=5, activation='relu')(input_layer)
    else:
      conv = L.Conv1D(filters=filter_means[i], kernel_size=9, padding='same', strides=5, activation='relu')(relu)
    drop = L.Dropout(0.25)(conv)
    conv = L.Conv1D(filters=filter_means[i], kernel_size=9, padding='same', strides=5)(drop)
    batch = L.BatchNormalization()(conv)
    #pool = L.MaxPool1D(2)(batch)
    relu = L.ReLU()(batch)
    drop = L.Dropout(0.25)(relu)
  #return L.Lambda(lambda x: L.K.batch_flatten(x))(relu)
  return L.Flatten()(drop)

In [6]:
def model_many_diseases2():
  input_layer = L.Input(shape=(4500, 12))
  input_layer2 = L.Input(shape=(4500, 4))
  input_layer3 = L.Input(shape=(60,))
  # inp
  model_12 = get_conv_model(filter_means[:2], input_layer)
  model_4 = get_conv_model(filter_means_2[:2], input_layer2)
  
  merged = L.Concatenate(axis=1)([model_12, model_4])
  merged2 = L.Concatenate(axis=1)([model_12, model_4, input_layer3])

  #dense_1_1 = L.Dense(1024, activation='relu')(merged)
  dense_1_2 = L.Dense(512, activation='relu')(merged)
  dense_2_2 = L.Dense(512, activation='relu')(merged)
  dense_3_2 = L.Dense(512, activation='relu')(merged)
  dense_4_2 = L.Dense(512, activation='relu')(merged)
  dense_5_2 = L.Dense(512, activation='relu')(merged)
  #dense_6_2 = L.Dense(512, activation='relu')(merged)
  dense_7_2 = L.Dense(572, activation='relu')(merged2)
  
  #dense_rhythm = L.Dense(4, activation='softmax')(dense_1_2)
  #dense_hyper = L.Dense(1, activation='sigmoid')(dense_2_2)
  
  dense_rhythm = L.Dense(4, activation='softmax')(dense_1_2)
  dense_hyper = L.Dense(1, activation='sigmoid')(dense_2_2)
  dense_extr = L.Dense(1, activation='sigmoid')(dense_3_2)
  dense_av = L.Dense(1, activation='sigmoid')(dense_4_2)
  dense_gis1 = L.Dense(1, activation='sigmoid')(dense_5_2)
  dense_gis2 = L.Dense(1, activation='sigmoid')(dense_5_2)
  dense_gis3 = L.Dense(1, activation='sigmoid')(dense_5_2)
  dense_gis4 = L.Dense(1, activation='sigmoid')(dense_5_2)
  dense_gis5 = L.Dense(1, activation='sigmoid')(dense_5_2)
  dense_gis6 = L.Dense(1, activation='sigmoid')(dense_5_2)
  #dense_isc = L.Dense(1, activation='sigmoid')(dense_6_2)
  dense_axis = L.Dense(9, activation='softmax')(dense_7_2)
  
  model = keras.Model([input_layer,input_layer2,input_layer3], [dense_rhythm,dense_hyper,dense_extr,dense_av,dense_gis1,dense_gis2,dense_gis3,
                                                      dense_gis4,dense_gis5,dense_gis6,dense_axis])
  return model

In [7]:
model = model_many_diseases2()

In [8]:
model.compile(loss=["sparse_categorical_crossentropy","binary_crossentropy","binary_crossentropy",
                            "binary_crossentropy","binary_crossentropy","binary_crossentropy",
                           "binary_crossentropy","binary_crossentropy","binary_crossentropy","binary_crossentropy",
                           "sparse_categorical_crossentropy"], optimizer=keras.optimizers.Adam(), metrics=['accuracy'])

In [9]:
def generator_st(x_train, y_train, batch_size, num_leads, out_neurons):
  # Create empty arrays to contain batch of features and labels#
  if (num_leads == 12):
    FLAG = 0
    batch_features = np.zeros((batch_size, 4500, 12))
  else:
    FLAG = 1
    batch_features_1 = np.zeros((batch_size, 4500, 12))
    batch_features_2 = np.zeros((batch_size, 4500, 4))
    batch_features_3 = np.zeros((batch_size, 60))
    
  if len(out_neurons) == 1:
    batch_labels = np.zeros((batch_size, out_neurons[0]))
  else:
    batch_labels = []
    for i in range(len(out_neurons)):
      batch_lab = np.zeros((batch_size, out_neurons[i]))
      batch_labels.append(batch_lab)
  
  if (FLAG == 0):
    while True:
      for i in range(batch_size):
          # choose random index in features
          index = np.random.choice(len(x_train), 1)[0]
          mu, sigma = 0, 0.5 
          noise = np.random.normal(mu, sigma, [3000,12])
          shift = np.random.randint(0, 501, 1)[0]
          batch_features[i] = x_train[index, shift:shift+3000, :] + noise 
          if len(out_neurons) == 1:
            batch_labels[i] = y_train[index]
          else:
            for j in range(len(out_neurons)):
              batch_labels[j][i] = y_train[j][index]
          
      yield batch_features, batch_labels
      
  elif (FLAG == 1):
    while True:
      for i in range(batch_size):
          # choose random index in features
          index = np.random.choice(len(x_train[0]), 1)[0]
          shift = np.random.randint(0, 501, 1)[0]
          mu, sigma = 0, 0.3
          noise = np.random.normal(mu, sigma, [4500,12])
          noise2 = np.random.normal(mu, sigma, [4500,4])

          batch_features_1[i] = x_train[0][index, shift:shift+4500]# + np.sin(noise)
          batch_features_2[i] = x_train[1][index, shift:shift+4500] 
          batch_features_3[i, :12] = batch_features_1[i].max(axis=0)
          batch_features_3[i, 12:24] = batch_features_1[i].min(axis=0)
          batch_features_3[i, 24:36] = batch_features_1[i].mean(axis=0)
          batch_features_3[i, 36:48] = batch_features_1[i].std(axis=0)
          batch_features_3[i, 48:] = batch_features_1[i].max(axis=0) + batch_features_1[i].min(axis=0)
          #batch_features_3[i] = x_train[2][index, :]
          if len(out_neurons) == 1:
            batch_labels[i] = y_train[index]
          else:
            for j in range(len(out_neurons)):
              batch_labels[j][i] = y_train[j][index]
            
      yield [batch_features_1, batch_features_2,batch_features_3], batch_labels

In [10]:
history = model.fit_generator(generator_st([x_train[:,:,:12],x_train[:,:,12:]], [y_train, y_train_hyper, y_train_extr, y_train_av, y_train_gis[:,0],
                                                                                y_train_gis[:,1] ,y_train_gis[:,2] ,y_train_gis[:,3], y_train_gis[:,4],
                                                                                y_train_gis[:,5] , y_train_axis], 128, 16, [1,1,1,1,1,1,1,1,1,1,1]), epochs=1, steps_per_epoch=8,
                                     validation_data=([x_test[:,0:4500,:12],x_test[:,0:4500,12:],x_test1], [y_test, y_test_hyper, y_test_extr, y_test_av, y_test_gis[:,0],
                                                                                y_test_gis[:,1] ,y_test_gis[:,2] ,y_test_gis[:,3], y_test_gis[:,4],
                                                                                y_test_gis[:,5] , y_test_axis]), class_weight = [{0:0.35010338, 1:1.66013072, 2:11.04347826, 3:2.21834061},
                                                                                                                                 {0:1.25925926, 1:0.82926829},
                                                                                                                                 {0:0.52211302, 1:11.80555556},
                                                                                                                                 {0:0.52696838, 1:9.77011494},
                                                                                                                                 {0:0.57744565, 1:3.72807018}, {0:0.50535077 ,1:47.22222222}, {0:5.00294291e-01 ,1:8.50000000e+02},
                                                                                                                                 {0:0.55230669 ,1:5.27950311}, {0:0.51924252 ,1:13.49206349}, {0:0.51515152 ,1:17},
                                                                                                                                 {0:1, 1:13.28125   ,  2:1.40728477,  3:0.37677305,  4:1.02163462,  5:0.4009434 , 6:3.86363636, 7:35.41666667,  8:1.25      }])

NameError: name 'x_train' is not defined